In [115]:
import sys
import argparse
import numpy as np
import time
import zipfile


def parse_reads_file(reads_fn):
    """
    :param reads_fn: the file containing all of the reads
    :return: outputs a list of all paired-end reads
    HINT: This might not work well if the number of reads is too large to handle in memory
    """
    try:
        with open(reads_fn, 'r') as rFile:
            print("Parsing Reads")
            first_line = True
            count = 0
            all_reads = []
            for line in rFile:
                count += 1
                if count % 1000 == 0:
                    print(count, " reads done")
                if first_line:
                    first_line = False
                    continue
                ends = line.strip().split(',')
                all_reads.append(ends)
        return all_reads
    except IOError:
        print("Could not read file: ", reads_fn)
        return None


def parse_ref_file(ref_fn):
    """
    :param ref_fn: the file containing the reference genome
    :return: a string containing the reference genome
    """
    try:
        with open(ref_fn, 'r') as gFile:
            print("Parsing Ref")
            first_line = True
            ref_genome = ''
            for line in gFile:
                if first_line:
                    first_line = False
                    continue
                ref_genome += line.strip()
        return ref_genome
    except IOError:
        print("Could not read file: ", ref_fn)
        return None
input_reads = parse_reads_file('reads_hw2undergrad_E_2_chr_1.txt')
reference = parse_ref_file('ref_hw2undergrad_E_2_chr_1.txt')

Parsing Reads
1000  reads done
2000  reads done
3000  reads done
4000  reads done
5000  reads done
6000  reads done
7000  reads done
8000  reads done
9000  reads done
10000  reads done
11000  reads done
12000  reads done
13000  reads done
14000  reads done
15000  reads done
16000  reads done
17000  reads done
18000  reads done
19000  reads done
20000  reads done
21000  reads done
22000  reads done
23000  reads done
24000  reads done
25000  reads done
26000  reads done
27000  reads done
28000  reads done
29000  reads done
30000  reads done
31000  reads done
32000  reads done
33000  reads done
34000  reads done
35000  reads done
36000  reads done
37000  reads done
38000  reads done
39000  reads done
40000  reads done
41000  reads done
42000  reads done
43000  reads done
44000  reads done
45000  reads done
46000  reads done
47000  reads done
48000  reads done
49000  reads done
50000  reads done
51000  reads done
52000  reads done
53000  reads done
54000  reads done
55000  reads done
56000

In [39]:
hash_table={}
for i in range(len(reference)-16):
    seq=reference[i:i+16]
    if seq not in hash_table:
        hash_table[seq]=[i]
    else:
        hash_table[seq]=hash_table[seq]+[i]
hash_table

{'CCCCGTTCGGGAGGTG': [0],
 'CCCGTTCGGGAGGTGG': [1],
 'CCGTTCGGGAGGTGGT': [2],
 'CGTTCGGGAGGTGGTC': [3],
 'GTTCGGGAGGTGGTCC': [4],
 'TTCGGGAGGTGGTCCC': [5],
 'TCGGGAGGTGGTCCCC': [6],
 'CGGGAGGTGGTCCCCA': [7],
 'GGGAGGTGGTCCCCAC': [8],
 'GGAGGTGGTCCCCACC': [9],
 'GAGGTGGTCCCCACCC': [10],
 'AGGTGGTCCCCACCCA': [11],
 'GGTGGTCCCCACCCAG': [12],
 'GTGGTCCCCACCCAGC': [13],
 'TGGTCCCCACCCAGCT': [14],
 'GGTCCCCACCCAGCTA': [15],
 'GTCCCCACCCAGCTAG': [16],
 'TCCCCACCCAGCTAGA': [17],
 'CCCCACCCAGCTAGAC': [18],
 'CCCACCCAGCTAGACC': [19],
 'CCACCCAGCTAGACCG': [20],
 'CACCCAGCTAGACCGT': [21],
 'ACCCAGCTAGACCGTT': [22],
 'CCCAGCTAGACCGTTT': [23],
 'CCAGCTAGACCGTTTA': [24],
 'CAGCTAGACCGTTTAA': [25],
 'AGCTAGACCGTTTAAT': [26],
 'GCTAGACCGTTTAATA': [27],
 'CTAGACCGTTTAATAA': [28],
 'TAGACCGTTTAATAAG': [29],
 'AGACCGTTTAATAAGA': [30],
 'GACCGTTTAATAAGAA': [31],
 'ACCGTTTAATAAGAAA': [32],
 'CCGTTTAATAAGAAAA': [33],
 'CGTTTAATAAGAAAAA': [34],
 'GTTTAATAAGAAAAAC': [35],
 'TTTAATAAGAAAAACT': [36],
 'TTAATAAGA

In [48]:
input_reads=input_reads[:150000]
seq_table=[]
for line in input_reads:
    st1=line[0]
    seq_table=seq_table+[[st1[0:16],st1[16:32],st1[32:48]]]
    st2=line[1]
    seq_table=seq_table+[[st2[0:16],st2[16:32],st2[32:48]]]
seq_table

[['AACACGGGTATTGTTA', 'ATACTATAAATAATTT', 'TGTTATATAAACCTTG'],
 ['CGCATGGCGGGGACTG', 'TGAAATGTCGTATTCA', 'GACCCATATGATGAGT'],
 ['TATCTGAGGCGGCTTG', 'TACGTATTCCTGTGCG', 'GCTCAAAGCGTCGCCG'],
 ['AGGACACGTATTGGGC', 'GGTGAACCATTTGTGT', 'AGTTTCTCAGTAGTTA'],
 ['TCAGCAAAGGGATGAC', 'GTGTCAGACGAGGCAT', 'ATTAATTCCTCACATT'],
 ['TGGTTGACAAAGATTC', 'TTAGACGTTGCTCGGT', 'ACAATTTTAGGTCAAG'],
 ['CGACCTGGACCATATG', 'ATCCTTGTGAGTAAGC', 'TGTGTTAAACCCAATC'],
 ['TTTTTCCTATGCTGGA', 'AGCCGACCTGCCTTCG', 'TGGAGTCGCGCCAAGT'],
 ['ACGTGATGAAATCGGA', 'TCATCAATCCCCTTTA', 'GATCCTAGGTGTCTAA'],
 ['TTACCTAATGCTCGCC', 'CTCCACGCGAGCGGGC', 'TGACCTTGGATACGAT'],
 ['TGGACTTCCCTCGTAC', 'TTAATCCCAGAGGTGT', 'GACATCGTAGTTCCCA'],
 ['TTAGTCGAGTACAGAG', 'ATGCCGACTAATATAG', 'AGAGATCTCCAATTAG'],
 ['TTCGGATTCTCCAGCT', 'TGTTTCGTCCTGGGGT', 'GGGGCAGGCTCCCTTG'],
 ['AGCGTTCCTCTACTTT', 'AGAGGAATTATTCTAC', 'CTGCGTTGAGGGGCCC'],
 ['GTTGGGACTGGACTCC', 'TGTATATGGAAATCTA', 'CCGTACGCCGCCCGTT'],
 ['CACCACCACTGCACGA', 'CGATCGGTATGTTTTC', 'GTTTCGCGTCAA

In [98]:
cand_snp=[]
for seqs in seq_table:
    cands=[]
    seq1=seqs[0]
    seq2=seqs[1]
    seq3=seqs[2]
    seq=seq1+seq2+seq3
    if seq1 in hash_table:
        inds=hash_table[seq1]
        for ind in inds:
            SNP_ind_list=[]
            if ind<=(1000000-48):
                for i in range(16,48):
                    compare_seq1=seq[i]
                    compare_seq2=reference[ind+i]
                    if compare_seq1!=compare_seq2:
                        SNP_ind_list=SNP_ind_list+[[compare_seq2,compare_seq1,ind+i]]
                    if len(SNP_ind_list)>2:
                        break
                if len(SNP_ind_list)<=2 and len(SNP_ind_list)>0:
                    cands=cands+SNP_ind_list
    if seq2 in hash_table:
        inds=hash_table[seq2]
        for ind in inds:
            SNP_ind_list=[]
            if ind>=16 and ind<=(1000000-32):
                for i in range(16):
                    compare_seq1=seq1[-i]
                    compare_seq2=reference[ind-i]
                    compare_seq3=seq3[i]
                    compare_seq4=reference[ind+i+16]
                    if compare_seq1!=compare_seq2:
                        SNP_ind_list=SNP_ind_list+[[compare_seq2,compare_seq1,ind-i]]
                    if compare_seq3!=compare_seq4:
                        SNP_ind_list=SNP_ind_list+[[compare_seq4,compare_seq3,ind+i+16]]
                    if len(SNP_ind_list)>2:
                        break
                if len(SNP_ind_list)<=2 and len(SNP_ind_list)>0:
                    cands=cands+SNP_ind_list
    if seq3 in hash_table:
        inds=hash_table[seq3]
        for ind in inds:
            SNP_ind_list=[]
            if ind>=32:
                seq12=seq1+seq2
                for i in range(32):
                    compare_seq1=seq12[-i]
                    compare_seq2=reference[ind-i]
                    if compare_seq1!=compare_seq2:
                        SNP_ind_list=SNP_ind_list+[[compare_seq2,compare_seq1,ind-i]]
                    if len(SNP_ind_list)>2:
                        break
                if len(SNP_ind_list)<=2 and len(SNP_ind_list)>0:
                    cands=cands+SNP_ind_list
    cands_temp=[]
    for cand in cands:
        if cand not in cands_temp:
            cands_temp=cands_temp+[cand]
    cand_snp=cand_snp+cands_temp
cand_snp

[['T', 'A', 757138],
 ['G', 'T', 204016],
 ['G', 'A', 365218],
 ['G', 'T', 365202],
 ['T', 'A', 700059],
 ['G', 'A', 700075],
 ['C', 'A', 241593],
 ['G', 'A', 123532],
 ['G', 'A', 185958],
 ['G', 'A', 195772],
 ['G', 'A', 264267],
 ['G', 'A', 324734],
 ['G', 'A', 486289],
 ['G', 'A', 904960],
 ['T', 'C', 30518],
 ['T', 'C', 178008],
 ['T', 'C', 229170],
 ['T', 'C', 425103],
 ['T', 'C', 523684],
 ['T', 'C', 527878],
 ['T', 'C', 811145],
 ['T', 'C', 904025],
 ['C', 'T', 30529],
 ['C', 'T', 178019],
 ['C', 'T', 229181],
 ['C', 'T', 425114],
 ['C', 'T', 523695],
 ['C', 'T', 527889],
 ['C', 'T', 811156],
 ['C', 'T', 904036],
 ['C', 'G', 313408],
 ['A', 'G', 313424],
 ['G', 'A', 885460],
 ['G', 'T', 885466],
 ['G', 'T', 750683],
 ['C', 'T', 750699],
 ['T', 'C', 702835],
 ['G', 'T', 702805],
 ['G', 'T', 874893],
 ['G', 'T', 251392],
 ['G', 'C', 599697],
 ['A', 'G', 850674],
 ['A', 'G', 850656],
 ['G', 'C', 873997],
 ['A', 'C', 874013],
 ['G', 'A', 408805],
 ['T', 'A', 408821],
 ['C', 'G', 930

In [86]:
cand_ins=[]
cand_del=[]
for seqs in seq_table:
    seq1=seqs[0]
    seq2=seqs[1]
    seq3=seqs[2]
    seq=seq1+seq2+seq3
    if seq1 in hash_table and seq3 in hash_table:
        inds1=hash_table[seq1]
        inds3=hash_table[seq3]
        for ind1 in inds1:
            for ind3 in inds3:
                if ind3-ind1==33:
                    refseq2=reference[(ind1+16):ind3]
                    pos=ind3-1
                    posi=16
                    for i in range(16):
                        if seq2[i]!=refseq2[i]:
                            pos=ind1+16+i
                            posi=i
                            break
                    allSame=True
                    for j in range(posi,16):
                        if seq2[j]!=refseq2[j+1]:
                            allSame=False
                    if allSame:
                        cand_del=cand_del+[[refseq2[posi],pos]]
                if ind3-ind1==31:
                    refseq2=reference[(ind1+16):ind3]
                    pos=ind3
                    posi=15
                    for i in range(15):
                        if seq2[i]!=refseq2[i]:
                            pos=ind1+16+i
                            posi=i
                            break
                    allSame=True
                    for j in range(posi,15):
                        if seq2[j+1]!=refseq2[j]:
                            allSame=False
                    if allSame:
                        cand_ins=cand_ins+[[(refseq2+seq3)[posi],pos]]
cand_del

[['C', 800251],
 ['C', 414291],
 ['G', 889753],
 ['A', 103730],
 ['G', 100394],
 ['G', 517696],
 ['C', 657516],
 ['C', 428937],
 ['T', 405657],
 ['T', 636875],
 ['G', 455293],
 ['G', 941777],
 ['A', 3673],
 ['T', 925745],
 ['G', 512867],
 ['G', 100394],
 ['T', 28299],
 ['A', 211931],
 ['T', 402074],
 ['C', 409123],
 ['G', 660843],
 ['A', 663061],
 ['T', 97042],
 ['C', 414291],
 ['T', 334489],
 ['A', 131813],
 ['C', 328524],
 ['C', 955277],
 ['G', 744067],
 ['A', 327905],
 ['C', 18954],
 ['T', 919993],
 ['A', 892218],
 ['A', 218189],
 ['C', 960436],
 ['G', 100394],
 ['A', 211931],
 ['T', 929015],
 ['C', 800251],
 ['T', 282200],
 ['A', 103730],
 ['A', 663060],
 ['C', 98066],
 ['G', 100394],
 ['C', 644557],
 ['G', 256518],
 ['G', 311172],
 ['T', 399109],
 ['C', 594149],
 ['G', 442438],
 ['T', 573219],
 ['A', 362294],
 ['C', 408753],
 ['C', 498372],
 ['C', 827552],
 ['A', 806273],
 ['C', 577649],
 ['T', 439546],
 ['C', 279383],
 ['C', 498372],
 ['T', 403880],
 ['A', 663061],
 ['T', 379476]

In [99]:
pos_count={}
for cand in cand_snp:
    pos=cand[2]
    if pos not in pos_count:
        pos_count[pos]=1
    else:
        pos_count[pos]+=1
pos_count

{757138: 1,
 204016: 1,
 365218: 1,
 365202: 1,
 700059: 1,
 700075: 2,
 241593: 1,
 123532: 1,
 185958: 1,
 195772: 1,
 264267: 1,
 324734: 1,
 486289: 1,
 904960: 1,
 30518: 2,
 178008: 2,
 229170: 2,
 425103: 2,
 523684: 2,
 527878: 2,
 811145: 2,
 904025: 2,
 30529: 1,
 178019: 1,
 229181: 1,
 425114: 1,
 523695: 1,
 527889: 1,
 811156: 1,
 904036: 1,
 313408: 1,
 313424: 1,
 885460: 2,
 885466: 1,
 750683: 1,
 750699: 1,
 702835: 2,
 702805: 1,
 874893: 2,
 251392: 1,
 599697: 1,
 850674: 2,
 850656: 1,
 873997: 1,
 874013: 1,
 408805: 2,
 408821: 2,
 930311: 1,
 930286: 2,
 56082: 1,
 171938: 1,
 428407: 1,
 584278: 1,
 736684: 1,
 943876: 1,
 954130: 1,
 958411: 1,
 984794: 1,
 56098: 2,
 171954: 2,
 428423: 2,
 584294: 2,
 736700: 2,
 943892: 2,
 954146: 2,
 958427: 2,
 984810: 2,
 141608: 1,
 141597: 9,
 141624: 1,
 296342: 3,
 296331: 1,
 745167: 2,
 745183: 2,
 360656: 1,
 360655: 1,
 644948: 1,
 644921: 1,
 460592: 1,
 460608: 1,
 359065: 1,
 397333: 1,
 632907: 1,
 837780:

In [113]:
ret=[]
for cand in cand_snp:
    pos=cand[2]
    if pos_count[pos]>=7:
        if cand not in ret:
            ret=ret+[cand]
len(ret)

2132

In [79]:
for i in range(4,5):
    print(7)

7


In [78]:
[2]+[5]

[2, 5]

In [ ]:
import sys
import argparse
import numpy as np
import time
import zipfile


def parse_reads_file(reads_fn):
    """
    :param reads_fn: the file containing all of the reads
    :return: outputs a list of all paired-end reads

    HINT: This might not work well if the number of reads is too large to handle in memory
    """
    try:
        with open(reads_fn, 'r') as rFile:
            print("Parsing Reads")
            first_line = True
            count = 0
            all_reads = []
            for line in rFile:
                count += 1
                if count % 1000 == 0:
                    print(count, " reads done")
                if first_line:
                    first_line = False
                    continue
                ends = line.strip().split(',')
                all_reads.append(ends)
        return all_reads
    except IOError:
        print("Could not read file: ", reads_fn)
        return None


def parse_ref_file(ref_fn):
    """
    :param ref_fn: the file containing the reference genome
    :return: a string containing the reference genome
    """
    try:
        with open(ref_fn, 'r') as gFile:
            print("Parsing Ref")
            first_line = True
            ref_genome = ''
            for line in gFile:
                if first_line:
                    first_line = False
                    continue
                ref_genome += line.strip()
        return ref_genome
    except IOError:
        print("Could not read file: ", ref_fn)
        return None


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='basic_hasher.py takes in data for homework assignment 2 consisting '
                                     'of a genome and a set of reads and aligns the reads to the reference genome, '
                                     'then calls SNPS and indels based on this alignment.')
    parser.add_argument('-g', '--referenceGenome', required=True, dest='reference_file',
                        help='File containing a reference genome.')
    parser.add_argument('-r', '--reads', required=True, dest='reads_file',
                        help='File containg sequencing reads.')
    parser.add_argument('-o', '--outputFile', required=True, dest='output_file',
                        help='Output file name.')
    parser.add_argument('-t', '--outputHeader', required=True, dest='output_header',
                        help='String that needs to be outputted on the first line of the output file so that the\n'
                             'online submission system recognizes which leaderboard this file should be submitted to.\n'
                             'This HAS to be one of the following:\n'
                             '1) practice_W_3_chr_1 for 10K length genome practice data\n'
                             '2) practice_E_1_chr_1 for 1 million length genome practice data\n'
                             '3) hw2undergrad_E_2_chr_1 for project 2 undergrad for-credit data\n'
                             '4) hw2grad_M_1_chr_1 for project 2 grad for-credit data\n')
    args = parser.parse_args()
    reference_fn = args.reference_file
    reads_fn = args.reads_file

    input_reads = parse_reads_file(reads_fn)
    if input_reads is None:
        sys.exit(1)
    reference = parse_ref_file(reference_fn)
    if reference is None:
        sys.exit(1)

    """
        TODO: Call functions to do the actual read alignment here

    """
    hash_table={}
    for i in range(len(reference)-16):
        seq=reference[i:i+16]
        if seq not in hash_table:
            hash_table[seq]=[i]
        else:
            hash_table[seq]=hash_table[seq]+[i]  #build the 16mer hashtables
    
    input_reads=input_reads[:150000]
    seq_table=[]
    for line in input_reads:
        st1=line[0]
        seq_table=seq_table+[[st1[0:16],st1[16:32],st1[32:48]]]
        st2=line[1]
        seq_table=seq_table+[[st2[0:16],st2[16:32],st2[32:48]]] #divide each read to 3 seqs
    
    cand_snp=[]
    for seqs in seq_table:
        cands=[]
        seq1=seqs[0]
        seq2=seqs[1]
        seq3=seqs[2]
        seq=seq1+seq2+seq3
        if seq1 in hash_table:
            inds=hash_table[seq1]
            for ind in inds:
                SNP_ind_list=[]
                if ind<=(1000000-48):
                    for i in range(16,48):
                        compare_seq1=seq[i]
                        compare_seq2=reference[ind+i]
                        if compare_seq1!=compare_seq2:
                            SNP_ind_list=SNP_ind_list+[[compare_seq2,compare_seq1,ind+i]]
                        if len(SNP_ind_list)>2:
                            break
                    if len(SNP_ind_list)<=2 and len(SNP_ind_list)>0:
                        cands=cands+SNP_ind_list
        if seq2 in hash_table:
            inds=hash_table[seq2]
            for ind in inds:
                SNP_ind_list=[]
                if ind>=16 and ind<=(1000000-32):
                    for i in range(16):
                        compare_seq1=seq1[-i]
                        compare_seq2=reference[ind-i]
                        compare_seq3=seq3[i]
                        compare_seq4=reference[ind+i+16]
                        if compare_seq1!=compare_seq2:
                            SNP_ind_list=SNP_ind_list+[[compare_seq2,compare_seq1,ind-i]]
                        if compare_seq3!=compare_seq4:
                            SNP_ind_list=SNP_ind_list+[[compare_seq4,compare_seq3,ind+i+16]]
                        if len(SNP_ind_list)>2:
                            break
                    if len(SNP_ind_list)<=2 and len(SNP_ind_list)>0:
                        cands=cands+SNP_ind_list
        if seq3 in hash_table:
            inds=hash_table[seq3]
            for ind in inds:
                SNP_ind_list=[]
                if ind>=32:
                    seq12=seq1+seq2
                    for i in range(32):
                        compare_seq1=seq12[-i]
                        compare_seq2=reference[ind-i]
                        if compare_seq1!=compare_seq2:
                            SNP_ind_list=SNP_ind_list+[[compare_seq2,compare_seq1,ind-i]]
                        if len(SNP_ind_list)>2:
                            break
                    if len(SNP_ind_list)<=2 and len(SNP_ind_list)>0:
                        cands=cands+SNP_ind_list       
        cands_temp=[]
        for cand in cands:
            if cand not in cands_temp:
                cands_temp=cands_temp+[cand]           #find all candidate snps
        cand_snp=cand_snp+cands_temp    
    
    cand_ins=[]
    cand_del=[]
    for seqs in seq_table:
        seq1=seqs[0]
        seq2=seqs[1]
        seq3=seqs[2]
        seq=seq1+seq2+seq3
        if seq1 in hash_table and seq3 in hash_table:
            inds1=hash_table[seq1]
            inds3=hash_table[seq3]
            for ind1 in inds1:
                for ind3 in inds3:
                    if ind3-ind1==33:
                        refseq2=reference[(ind1+16):ind3]
                        pos=ind3-1
                        posi=16
                        for i in range(16):
                            if seq2[i]!=refseq2[i]:
                                pos=ind1+16+i
                                posi=i
                                break
                        allSame=True
                        for j in range(posi,16):
                            if seq2[j]!=refseq2[j+1]:
                                allSame=False
                        if allSame:
                            cand_del=cand_del+[[refseq2[posi],pos]]
                    if ind3-ind1==31:
                        refseq2=reference[(ind1+16):ind3]
                        pos=ind3
                        posi=15
                        for i in range(15):
                            if seq2[i]!=refseq2[i]:
                                pos=ind1+16+i
                                posi=i
                                break
                        allSame=True
                        for j in range(posi,15):
                            if seq2[j+1]!=refseq2[j]:    #find indels
                                allSame=False
                        if allSame:
                            cand_ins=cand_ins+[[(refseq2+seq3)[posi],pos]]
    pos_count={}
    for cand in cand_snp:
        pos=cand[2]
        if pos not in pos_count:
            pos_count[pos]=1
        else:
            pos_count[pos]+=1    #count number of apperance of each snp
    
    ret=[]
    for cand in cand_snp:
        pos=cand[2]
        if pos_count[pos]>=6:
            if cand not in ret:
                ret=ret+[cand]
    
    snps = ret
    insertions = cand_ins
    deletions = cand_del

    output_fn = args.output_file
    zip_fn = output_fn + '.zip'
    with open(output_fn, 'w') as output_file:
        output_file.write('>' + args.output_header + '\n>SNP\n')
        for x in snps:
            output_file.write(','.join([str(u) for u in x]) + '\n')
        output_file.write('>INS\n')
        for x in insertions:
            output_file.write(','.join([str(u) for u in x]) + '\n')
        output_file.write('>DEL\n')
        for x in deletions:
            output_file.write(','.join([str(u) for u in x]) + '\n')
    with zipfile.ZipFile(zip_fn, 'w') as myzip:
        myzip.write(output_fn)


In [111]:
print('finished snp list')

finished snp list


In [114]:
print('finished finding snp candidates')

finished finding snp candidates


In [116]:
#try divide to two seqs

In [117]:
hash_table={}
for i in range(len(reference)-25):
    seq=reference[i:i+25]
    if seq not in hash_table:
        hash_table[seq]=[i]
    else:
        hash_table[seq]=hash_table[seq]+[i]
hash_table

{'CCCCGTTCGGGAGGTGGTCCCCACC': [0],
 'CCCGTTCGGGAGGTGGTCCCCACCC': [1],
 'CCGTTCGGGAGGTGGTCCCCACCCA': [2],
 'CGTTCGGGAGGTGGTCCCCACCCAG': [3],
 'GTTCGGGAGGTGGTCCCCACCCAGC': [4],
 'TTCGGGAGGTGGTCCCCACCCAGCT': [5],
 'TCGGGAGGTGGTCCCCACCCAGCTA': [6],
 'CGGGAGGTGGTCCCCACCCAGCTAG': [7],
 'GGGAGGTGGTCCCCACCCAGCTAGA': [8],
 'GGAGGTGGTCCCCACCCAGCTAGAC': [9],
 'GAGGTGGTCCCCACCCAGCTAGACC': [10],
 'AGGTGGTCCCCACCCAGCTAGACCG': [11],
 'GGTGGTCCCCACCCAGCTAGACCGT': [12],
 'GTGGTCCCCACCCAGCTAGACCGTT': [13],
 'TGGTCCCCACCCAGCTAGACCGTTT': [14],
 'GGTCCCCACCCAGCTAGACCGTTTA': [15],
 'GTCCCCACCCAGCTAGACCGTTTAA': [16],
 'TCCCCACCCAGCTAGACCGTTTAAT': [17],
 'CCCCACCCAGCTAGACCGTTTAATA': [18],
 'CCCACCCAGCTAGACCGTTTAATAA': [19],
 'CCACCCAGCTAGACCGTTTAATAAG': [20],
 'CACCCAGCTAGACCGTTTAATAAGA': [21],
 'ACCCAGCTAGACCGTTTAATAAGAA': [22],
 'CCCAGCTAGACCGTTTAATAAGAAA': [23],
 'CCAGCTAGACCGTTTAATAAGAAAA': [24],
 'CAGCTAGACCGTTTAATAAGAAAAA': [25],
 'AGCTAGACCGTTTAATAAGAAAAAC': [26],
 'GCTAGACCGTTTAATAAGAAAAACT': [27],
 '

In [130]:
cand_snp=[]
for line in input_reads:
    for seq in line:
        seq1=seq[0:25]
        seq2=seq[25:50]
        if seq1 in hash_table:
            inds=hash_table[seq1]
            for ind in inds:
                SNP_ind_list=[]
                if ind<=(1000000-50):
                    for i in range(25,50):
                        compare_seq1=seq[i]
                        compare_seq2=reference[ind+i]
                        if compare_seq1!=compare_seq2:
                            SNP_ind_list=SNP_ind_list+[[compare_seq2,compare_seq1,ind+i]]
                        if len(SNP_ind_list)>2:
                            break
                    if len(SNP_ind_list)<=2 and len(SNP_ind_list)>0:
                        cand_snp=cand_snp+SNP_ind_list
        if seq2 in hash_table:
            inds=hash_table[seq2]
            for ind in inds:
                SNP_ind_list=[]
                if ind>=25:
                    for i in range(25):
                        compare_seq1=seq1[-i]
                        compare_seq2=reference[ind-i]
                        if compare_seq1!=compare_seq2:
                            SNP_ind_list=SNP_ind_list+[[compare_seq2,compare_seq1,ind-i]]
                        if len(SNP_ind_list)>2:
                            break
                    if len(SNP_ind_list)<=2 and len(SNP_ind_list)>0:
                        cand_snp=cand_snp+SNP_ind_list

In [155]:
pos_count={}
for cand in cand_snp:
    pos=cand[2]
    var=cand[1]
    comb=(pos,var)
    if comb not in pos_count:
        pos_count[comb]=1
    else:
        pos_count[comb]+=1
ret=[]
for cand in cand_snp:
    pos=cand[2]
    var=cand[1]
    comb=(pos,var)
    if pos_count[comb]>=6:
        if cand not in ret:
            ret=ret+[cand]
len(ret)

4874

In [ ]:
ret=[]
for cand in cand_snp:
    pos=cand[2]
    if pos_count[pos]>=10:
        if cand not in ret:
            ret=ret+[cand]
len(ret)

In [ ]:
import sys
import argparse
import numpy as np
import time
import zipfile


def parse_reads_file(reads_fn):
    """
    :param reads_fn: the file containing all of the reads
    :return: outputs a list of all paired-end reads

    HINT: This might not work well if the number of reads is too large to handle in memory
    """
    try:
        with open(reads_fn, 'r') as rFile:
            print("Parsing Reads")
            first_line = True
            count = 0
            all_reads = []
            for line in rFile:
                count += 1
                if count % 1000 == 0:
                    print(count, " reads done")
                if first_line:
                    first_line = False
                    continue
                ends = line.strip().split(',')
                all_reads.append(ends)
        return all_reads
    except IOError:
        print("Could not read file: ", reads_fn)
        return None


def parse_ref_file(ref_fn):
    """
    :param ref_fn: the file containing the reference genome
    :return: a string containing the reference genome
    """
    try:
        with open(ref_fn, 'r') as gFile:
            print("Parsing Ref")
            first_line = True
            ref_genome = ''
            for line in gFile:
                if first_line:
                    first_line = False
                    continue
                ref_genome += line.strip()
        return ref_genome
    except IOError:
        print("Could not read file: ", ref_fn)
        return None


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='basic_hasher.py takes in data for homework assignment 2 consisting '
                                     'of a genome and a set of reads and aligns the reads to the reference genome, '
                                     'then calls SNPS and indels based on this alignment.')
    parser.add_argument('-g', '--referenceGenome', required=True, dest='reference_file',
                        help='File containing a reference genome.')
    parser.add_argument('-r', '--reads', required=True, dest='reads_file',
                        help='File containg sequencing reads.')
    parser.add_argument('-o', '--outputFile', required=True, dest='output_file',
                        help='Output file name.')
    parser.add_argument('-t', '--outputHeader', required=True, dest='output_header',
                        help='String that needs to be outputted on the first line of the output file so that the\n'
                             'online submission system recognizes which leaderboard this file should be submitted to.\n'
                             'This HAS to be one of the following:\n'
                             '1) practice_W_3_chr_1 for 10K length genome practice data\n'
                             '2) practice_E_1_chr_1 for 1 million length genome practice data\n'
                             '3) hw2undergrad_E_2_chr_1 for project 2 undergrad for-credit data\n'
                             '4) hw2grad_M_1_chr_1 for project 2 grad for-credit data\n')
    args = parser.parse_args()
    reference_fn = args.reference_file
    reads_fn = args.reads_file

    input_reads = parse_reads_file(reads_fn)
    if input_reads is None:
        sys.exit(1)
    reference = parse_ref_file(reference_fn)
    if reference is None:
        sys.exit(1)

    """
        TODO: Call functions to do the actual read alignment here

    """
    hash_table={}
    for i in range(len(reference)-25):
        seq=reference[i:i+25]
        if seq not in hash_table:
            hash_table[seq]=[i]
        else:
            hash_table[seq]=hash_table[seq]+[i]  #build the 25mer hashtables
    
    cand_snp=[]
    for line in input_reads:
        for seq in line:
            seq1=seq[0:25]
            seq2=seq[25:50]
            if seq1 in hash_table:
                inds=hash_table[seq1]
                for ind in inds:
                    SNP_ind_list=[]
                    if ind<=(1000000-50):
                        for i in range(25,50):
                            compare_seq1=seq[i]
                            compare_seq2=reference[ind+i]
                            if compare_seq1!=compare_seq2:
                                SNP_ind_list=SNP_ind_list+[[compare_seq2,compare_seq1,ind+i]]
                            if len(SNP_ind_list)>2:
                                break
                        if len(SNP_ind_list)<=2 and len(SNP_ind_list)>0:
                            cand_snp=cand_snp+SNP_ind_list
            if seq2 in hash_table:
                inds=hash_table[seq2]
                for ind in inds:
                    SNP_ind_list=[]
                    if ind>=25:
                        for i in range(25):
                            compare_seq1=seq1[-i]
                            compare_seq2=reference[ind-i]
                            if compare_seq1!=compare_seq2:
                                SNP_ind_list=SNP_ind_list+[[compare_seq2,compare_seq1,ind-i]]
                            if len(SNP_ind_list)>2:
                                break
                        if len(SNP_ind_list)<=2 and len(SNP_ind_list)>0:
                            cand_snp=cand_snp+SNP_ind_list
    print('finish candidate snps')
    pos_count={}
    for cand in cand_snp:
        pos=cand[2]
        var=cand[1]
        comb=(pos,var)
        if comb not in pos_count:
            pos_count[comb]=1
        else:
            pos_count[comb]+=1
    ret=[]
    for cand in cand_snp:
        pos=cand[2]
        var=cand[1]
        comb=(pos,var)
        if pos_count[comb]>=8:
            if cand not in ret:
                ret=ret+[cand]  #finish snp part
    print('finish snp')
    print(len(ret))
    hash_table={}
    for i in range(len(reference)-16):
        seq=reference[i:i+16]
        if seq not in hash_table:
            hash_table[seq]=[i]
        else:
            hash_table[seq]=hash_table[seq]+[i]   #build the 16mer hashtables
    
    cand_ins=[]
    cand_del=[]
    for line in input_reads:
        for seq in line:
            seq1=seq[0:16]
            seq2=seq[16:32]
            seq3=seq[32:48]
            if seq1 in hash_table and seq3 in hash_table:
                inds1=hash_table[seq1]
                inds3=hash_table[seq3]
                for ind1 in inds1:
                    for ind3 in inds3:
                        if ind3-ind1==33:
                            refseq2=reference[(ind1+16):ind3]
                            pos=ind3-1
                            posi=16
                            for i in range(16):
                                if seq2[i]!=refseq2[i]:
                                    pos=ind1+16+i
                                    posi=i
                                    break
                            allSame=True
                            for j in range(posi,16):
                                if seq2[j]!=refseq2[j+1]:
                                    allSame=False
                            if allSame:
                                cand_del=cand_del+[[refseq2[posi],pos]]
                        if ind3-ind1==31:
                            refseq2=reference[(ind1+16):ind3]
                            pos=ind3
                            posi=15
                            for i in range(15):
                                if seq2[i]!=refseq2[i]:
                                    pos=ind1+16+i
                                    posi=i
                                    break
                            allSame=True
                            for j in range(posi,15):
                                if seq2[j+1]!=refseq2[j]:    #find indels
                                    allSame=False
                            if allSame:
                                cand_ins=cand_ins+[[(refseq2+seq3)[posi],pos]]
    
    snps = ret
    insertions = cand_ins
    deletions = cand_del

    output_fn = args.output_file
    zip_fn = output_fn + '.zip'
    with open(output_fn, 'w') as output_file:
        output_file.write('>' + args.output_header + '\n>SNP\n')
        for x in snps:
            output_file.write(','.join([str(u) for u in x]) + '\n')
        output_file.write('>INS\n')
        for x in insertions:
            output_file.write(','.join([str(u) for u in x]) + '\n')
        output_file.write('>DEL\n')
        for x in deletions:
            output_file.write(','.join([str(u) for u in x]) + '\n')
    with zipfile.ZipFile(zip_fn, 'w') as myzip:
        myzip.write(output_fn)